In [1]:
from import_libraries.project_functions import *

# warnings.filterwarnings("ignore")

# data from MT5

In [2]:
# symbol = 'EURUSD.pro'
data = get_historical_data() # D1
# data_m5 = get_historical_data(timeframe=mt5.TIMEFRAME_M5, symbol="GER30", start='2022-01-01')
data.shape

Połączono z kontem


(3666, 12)

# experiments for manual selection of lines

In [3]:
# rebound_best_parameters = optimize_parameters(data, rebound_analysis, calculate_accumulated_price_changes,
#                                               windows_size= [i for i in range(1, 366, 1)],
#                                               bias= [i for i in range(1, 50, 1)]
#                                                 )

# print(f"Najlepsze opcje: {rebound_best_parameters}")
rebound_best_parameters = {'window_size': 2, 'bias': 10}

# Najlepsze opcje: {'window_size': 2, 'bias': 10}

In [4]:
current_data_optimize = define_level(data, rebound_best_parameters["window_size"] ,
                                  rebound_best_parameters["bias"])

rebound_data = rebound_analysis(current_data_optimize)
current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
current_score

C:\Users\Alex\Desktop\Data Science\SDA\currency_exchange\function\preprocess_function.py:413: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'sell' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



(8340.900000000009, 2724.3000000000093)

In [5]:
sum(current_score) / (len(data)  /(22 * 12 )) / 12

66.40327332242238

# Experiments with Genetic Algorithm for line

In [6]:
# class GeneticAlgorithm:
#     """
#     A class representing a Genetic Algorithm for parameter optimization in trading strategies.

#     Attributes:
#         data (pd.DataFrame): The financial data used for analysis.
#         analysis (callable): The analysis function to process the data.
#         calculate (callable): The scoring function to evaluate the performance of parameter combinations.
#         window_sizes (list): Possible values for the window size parameter.
#         biases (list): Possible values for the bias parameter.
#         best_individual (list): The best parameter combination found by the genetic algorithm.

#     Methods:
#         init_individual(individual_class): Initializes an individual for the genetic algorithm.
#         evaluate(individual): Evaluates the fitness of an individual based on the provided scoring function.
#         mutate(individual): Performs mutation on an individual to introduce diversity in the population.
#         calculate_parameters(params): Calculates the score for a given set of parameters.
#         run_genetic_algorithm(population_size, offspring_size, cx_probability, mut_probability, n_generations):
#             Runs the genetic algorithm to find the best parameter combination.
#         get_best_individual(): Returns the best parameter combination found by the genetic algorithm.
#     """
    
#     def __init__(self, data, analysis, calculate, window_sizes, biases=[1]):
#         """Initializes the GeneticAlgorithm instance.

#         Args:
#             data (pd.DataFrame): The financial data used for analysis.
#             analysis (callable): The analysis function to process the data.
#             calculate (callable): The scoring function to evaluate the performance of parameter combinations.
#             window_sizes (list): Possible values for the window size parameter.
#             biases (list, optional): Possible values for the bias parameter. Defaults to [1].
#         """
#         # Initialize the GeneticAlgorithm instance with the provided parameters
#         # Attribute Initialization:
#         self.data = data
#         self.analysis = analysis
#         self.calculate = calculate
#         self.window_sizes = window_sizes
#         self.biases = biases
#         self.best_individual = None

#         # Problem Definition
#         # Define the problem as a maximization problem
#         creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#         creator.create("Individual", list, fitness=creator.FitnessMax)

#         # Toolbox Initialization
#         # Create a toolbox with the necessary components
#         self.toolbox = base.Toolbox()

#         # Registering Functions in the Toolbox
#         # Register an initialization method for individuals
#         self.toolbox.register("individual", self.init_individual, creator.Individual)
        
#         # Register a method to initialize a population of individuals
#         self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
#         # Register the evaluation method for individuals
#         self.toolbox.register("evaluate", self.evaluate)
        
#         # Register the crossover method using Blend Crossover with a specified alpha value
#         self.toolbox.register("mate", tools.cxBlend, alpha=0.5)
        
#         # Register the mutation method
#         self.toolbox.register("mutate", self.mutate)
        
#         # Register the selection method using Tournament Selection with a tournament size of 3
#         self.toolbox.register("select", tools.selTournament, tournsize=3)

    
#     def init_individual(self, individual_class):
#         """
#         Initializes an individual for the genetic algorithm.

#         Args:
#             individual_class: The class representing an individual in the genetic algorithm.

#         Returns:
#             list: The initialized individual.
#         """
        
#         return individual_class([random.choice(self.window_sizes), random.choice(self.biases)])

    
#     def evaluate(self, individual):
#         """
#         Evaluates the fitness of an individual based on the provided scoring function.

#         Args:
#             individual (list): The individual representing a parameter combination.

#         Returns:
#             tuple: The fitness score of the individual.
#         """
        
#         window_size, bias = map(int, individual)
#         params = {'window_size': window_size, 'bias': bias}
#         score = self.calculate_parameters(params)
#         return (score,)

    
#     def mutate(self, individual):
#         """
#         Performs mutation on an individual to introduce diversity in the population.

#         Args:
#             individual (list): The individual to be mutated.

#         Returns:
#             tuple: The mutated individual.
#         """
        
#         if random.random() < 0.5:
#             individual[0] = int(abs(individual[0] + random.randint(-5, 5)))
#         else:
#             individual[1] = random.choice(self.biases)
#         return individual,

    
#     def calculate_parameters(self, params):
#         """
#         Calculates the score for a given set of parameters.

#         Args:
#             params (dict): The parameter values.

#         Returns:
#             float: The calculated score.
#         """
        
#         current_data = define_level(self.data, params['window_size'], params['bias'])
#         rebound_data = self.analysis(current_data)
#         return self.calculate(rebound_data)

        
#     def run_genetic_algorithm(self, population_size=10, offspring_size=50, cx_probability=0.7, mut_probability=0.2, n_generations=10):
#         """
#         Runs the genetic algorithm to find the best parameter combination.

#         Args:
#             population_size (int, optional): The size of the initial population. Defaults to 10.
#             offspring_size (int, optional): The size of the offspring population. Defaults to 50.
#             cx_probability (float, optional): The crossover probability. Defaults to 0.7.
#             mut_probability (float, optional): The mutation probability. Defaults to 0.2.
#             n_generations (int, optional): The number of generations. Defaults to 10.

#         Returns:
#             list: The best parameter combination found by the genetic algorithm.
#         """
#         # Create an initial population
#         population = self.toolbox.population(n=population_size)

#         # Run the genetic algorithm
#         algorithms.eaMuPlusLambda(population, self.toolbox, mu=population_size, lambda_=offspring_size,
#                                   cxpb=cx_probability, mutpb=mut_probability, ngen=n_generations, stats=None, halloffame=None)

#         # Get the best individual
#         self.best_individual = tools.selBest(population, k=1)[0]
#         print("Best Parameters:", self.best_individual)

#         return self.best_individual

    
#     def get_best_individual(self):
#         """
#         Returns the best parameter combination found by the genetic algorithm.

#         Returns:
#             list: The best parameter combination.
#         """
#         return self.best_individual


# # Example usage of the GeneticAlgorithm class
# # Create an instance of the class
# genetic_algorithm = GeneticAlgorithm(data, rebound_analysis, calculate_accumulated_price_changes, 
#                                      window_sizes= [i for i in range(1, 366, 1)], 
#                                      biases= [i for i in range(1, 50, 1)])


# # Run the genetic algorithm
# best_individual = genetic_algorithm.run_genetic_algorithm(population_size=10, 
#                                                           offspring_size=100, 
#                                                           cx_probability=0.7, 
#                                                           mut_probability=0.2, 
#                                                           n_generations=50)

In [7]:
# # Create an instance of the class
# genetic_algorithm = GeneticAlgorithm(data, rebound_analysis, calculate_accumulated_price_changes, 
#                                      window_sizes= [i for i in range(1, 366, 1)], 
#                                      biases= [i for i in range(1, 50, 1)])


# # Run the genetic algorithm
# best_individual = genetic_algorithm.run_genetic_algorithm(population_size=10, 
#                                                           offspring_size=100, 
#                                                           cx_probability=0.7, 
#                                                           mut_probability=0.2, 
#                                                           n_generations=50)

In [8]:
# current_data_algorithm = define_level(data,
#                 ceil(genetic_algorithm.get_best_individual()[0]),
#                 ceil(genetic_algorithm.get_best_individual()[1]) )

# rebound_data = rebound_analysis(current_data_algorithm)
# current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
# current_score

In [9]:
# sum(current_score) / (data(data)  /(22 * 12 )) / 12


# Order

In [10]:
# automated_trading_from_signals(rebound_data, symbol= symbol)

# data preprocessing with class Preprocessing_stock_data


In [11]:
del data
data = get_historical_data() # D1

Połączono z kontem


In [12]:
columns_name = {'Open':'open', 'High' : 'high' , 'Low' : 'low', 
                'Close' : 'close', "Volume":"volume" }
data = data.rename(columns= columns_name)
data.sample(5)

,Date,open,high,low,close,Tick_volume,Spread,Real_volume,volume,MaxPositivePriceChange,MaxNegativePriceChange,PriceChange
356,2010-11-17,6674.9,6712.5,6659.5,6703.0,780,0,0,53.0,37.6,15.4,29.5
538,2011-08-04,6700.7,6749.7,6271.9,6276.5,5315,0,0,477.8,49.0,428.8,437.2
2692,2020-02-04,13130.5,13291.4,13102.4,13277.6,89612,9,0,189.0,160.9,28.1,252.7
535,2011-08-01,7265.7,7298.5,6947.0,6989.0,4264,0,0,351.5,32.8,318.7,164.4
542,2011-08-10,5990.5,6117.7,5542.4,5610.4,7706,0,0,575.3,127.2,448.1,400.3


In [13]:
preprocessor = Preprocessing_stock_data(data)

data_indicators_pattern  =  preprocessor.add_indicators_pattern_recognition_functions()
data_calculate_overlap_studies  =  preprocessor.calculate_overlap_studies()
data_math_operator_functions  =  preprocessor.math_operator_functions()
data_math_transform_functions  =  preprocessor.math_transform_functions()
data_momentum_indicator_functions  =  preprocessor.momentum_indicator_functions()
data_statistic =  preprocessor.statistic_functions()


In [14]:
print(data_indicators_pattern.shape)
print(data_calculate_overlap_studies.shape)
print(data_math_operator_functions.shape)
print(data_math_operator_functions.shape)
print(data_math_transform_functions.shape)
print(data_momentum_indicator_functions.shape)
print(data_statistic.shape)

(3666, 66)
(3666, 38)
(3666, 23)
(3666, 23)
(3666, 20)
(3666, 65)
(3666, 41)


In [15]:
data_indicators_pattern.sample(5)

,open,high,low,close,volume,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,...,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
1512,11019.4,11071.4,10863.6,10962.6,207.8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,9641.3,9683.5,9537.0,9671.3,146.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3040,15750.0,15793.0,15712.0,15745.0,81.0,0,0,0,0,0,...,-100,0,0,0,0,0,0,0,0,0
159,5568.0,5584.0,5455.5,5511.5,128.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1398,9673.7,9849.8,9651.3,9849.0,198.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# experiments with RFELSTM and data from Preprocessing_stock_data

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Attention
from tensorflow.compat.v1.train import Optimizer
from tensorflow.keras.models import clone_model
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tqdm.notebook import tqdm

import seaborn as sb
import matplotlib.pyplot as plt

In [20]:
data_indicators_pattern.sample(5)

,open,high,low,close,volume,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,...,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
1595,9576.0,9680.3,9390.3,9679.3,290.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
387,7000.5,7041.3,6976.7,6986.2,64.6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214,15433.0,15533.2,15315.2,15424.2,218.0,0,0,0,0,100,...,-100,0,0,0,0,0,0,0,0,0
2961,13943.0,13971.0,13790.0,13910.0,181.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100
252,6224.0,6251.5,6102.0,6124.5,149.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## experiments № 1

In [140]:
%%time
class AttentionRFE:
    def __init__(self, data):
        self.data = data
        self.X = None
        self.y = None
        self.model = None

    def prepare_data(self, window_size=25, y_column="close"):
        X, y = [], []
    
        for i in range(len(self.data) - window_size):
            window = self.data.iloc[i:(i + window_size)].drop(y_column, axis=1)
            X.append(window.values)
            y.append(self.data.loc[i + window_size, y_column])
    
        self.X = np.array(X)
        self.y = np.array(y)
    
        return self.X, self.y

    def model_LSTM(self, shape: list[int, int]):
        try:
            input_layer = Input(shape = shape )
            
            lstm_layer = LSTM(64, activation='relu', return_sequences=True)(input_layer)
            output_tensor, weights = Attention()([lstm_layer, lstm_layer], return_attention_scores=True)
            flatten_layer = Flatten()(output_tensor)
            output_layer = Dense(1, activation='linear',)(flatten_layer)
    
            model = Model(inputs=input_layer, outputs=output_layer)
            # model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    
            self.model = model
            return self.model
    
        except AttributeError as e:
            print(e)
            if self.X is None or self.y is None:
                self.prepare_data()
            return self.model_LSTM()

    def fit(self, X=None, y=None, num_features_to_keep=1, window_size=25, epochs=10):
        
        if not X and not y:
            self.prepare_data(window_size)
        else:
            self.X = X
            self.y = y

    
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
    
        for _ in tqdm(range(self.X.shape[2] - num_features_to_keep)):
            
            model = clone_model(self.model_LSTM( shape=[X_train.shape[1], X_train.shape[2]]))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    
            # Обучение модели на текущих данных
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
    
            # Оценка модели на тестовых данных до обучения
            y_pred_before = model.predict(X_test)
            mse_before = mean_squared_error(y_test, y_pred_before)
    
            # Получение весов из слоя Attention
            attention_layer = model.layers[2]
    
            # Предполагая, что attention_layer - это MultiHeadAttention слой из TensorFlow
            _, attention_scores = attention_layer([X_test[:1], X_train[:1]], return_attention_scores=True)
            print(attention_scores)

    
            # Суммирование весов по признакам
            feature_importance = tf.reduce_sum(attention_scores, axis=2)
            print(feature_importance)
    
    
            # Находим индекс признака с наименьшим весом
            idx_to_remove = np.argmin(feature_importance)
            # print(idx_to_remove)
    
            # Удаляем признак с наименьшим весом из данных
            X_train = np.delete(X_train, idx_to_remove, axis=2)
            X_test = np.delete(X_test, idx_to_remove, axis=2)
            
    
            # Обновление модели
            model = clone_model(self.model_LSTM([X_train.shape[1], X_train.shape[2]] ))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
            
            # Обучение модели на новых данных
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
    
            # Оценка модели на тестовых данных после обучения
            y_pred_after = model.predict(X_test)
            mse_after = mean_squared_error(y_test, y_pred_after)
    
            print(f"Removed feature at index {idx_to_remove}. Test MSE before training: {mse_before}. Test MSE after training: {mse_after}")


# Создание экземпляра класса AttentionRFE
rfe = AttentionRFE(data_indicators_pattern)
rfe.fit()

  0%|          | 0/64 [00:00<?, ?it/s]

23/23 [==============================] - 0s 5ms/step
tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

KeyboardInterrupt: 

## experiments GradientRFE № 2

Nasza warstwa Attention nie zachowuje się zbyt dziwnie i nie zwraca oczekiwanych danych.

"Problem z pierwszą częścią polega na tym, że usuwa tylko jedną cechę, a następnie ponownie przeprowadza wyszukiwanie i usuwa inną, i tak dalej do n. Jednak co, jeśli po połączeniu słabej cechy, która została wcześniej usunięta, z tymi, które pozostały po n iteracjach, wynik okazałby się lepszy? Konieczne jest opracowanie metody łączenia parametrów. Jednak implementacja czegoś takiego byłaby bardzo nieoptymalna, ponieważ czas wyszukiwania najlepszych parametrów wyniósłby n!, gdzie n to liczba cech. Spróbujmy podejść do zadania z innej strony"

In [ ]:
# from itertools import combinations

# # Ваш список чисел
# numbers = [1, 2, 3]

# # Создание всех возможных комбинаций
# all_combinations = []
# for r in range(1, len(numbers) + 1):
#     all_combinations.extend(combinations(numbers, r))

# print(all_combinations)


In [ ]:
%%time
class GradientRFE:
    def __init__(self, data):
        self.data = data
        self.X = None
        self.y = None
        self.model = None

    def prepare_data(self, window_size=25, y_column="close"):
        X, y = [], []
    
        for i in range(len(self.data) - window_size):
            window = self.data.iloc[i:(i + window_size)].drop(y_column, axis=1)
            X.append(window.values)
            y.append(self.data.loc[i + window_size, y_column])
    
        self.X = np.array(X)
        self.y = np.array(y)
    
        return self.X, self.y

    def model_LSTM(self, shape: list[int, int]):
        try:
            input_layer = Input(shape = shape )
            
            lstm_layer = LSTM(64, activation='relu', return_sequences=True)(input_layer)
            output_tensor, weights = Attention()([lstm_layer, lstm_layer], return_attention_scores=True)
            flatten_layer = Flatten()(output_tensor)
            output_layer = Dense(1, activation='linear',)(flatten_layer)
    
            model = Model(inputs=input_layer, outputs=output_layer)
    
            self.model = model
            
            return self.model
    
        except AttributeError as e:
            print(e)
            if self.X is None or self.y is None:
                self.prepare_data()
            return self.model_LSTM()

            
    def fit(self, X=None, y=None, num_features_to_keep=1, window_size=25, epochs=10):
        
        if not X and not y:
            self.prepare_data(window_size)
        else:
            self.X = X
            self.y = y

    
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
    
        for _ in tqdm(range(self.X.shape[2] - num_features_to_keep)):
            
            model = clone_model(self.model_LSTM( shape=[X_train.shape[1], X_train.shape[2]]))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    
            # Обучение модели на текущих данных
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
            
            # # Оценка модели на тестовых данных до обучения
            # y_pred_before = model.predict(X_test)
            # mse_before = mean_squared_error(y_test, y_pred_before)
    
            # # Получение весов из слоя Attention
            # attention_layer = model.layers[2]
    
            # # Предполагая, что attention_layer - это MultiHeadAttention слой из TensorFlow
            # _, attention_scores = attention_layer([X_test[:1], X_train[:1]], return_attention_scores=True)
            # # print(attention_scores)
    
            # # Суммирование весов по признакам
            # feature_importance = tf.reduce_sum(attention_scores, axis=2)
            # # print(feature_importance)
    
    
            # # Находим индекс признака с наименьшим весом
            # idx_to_remove = np.argmin(feature_importance)
            # print(idx_to_remove)

            
    
            # Удаляем признак с наименьшим весом из данных
            X_train = np.delete(X_train, idx_to_remove, axis=2)
            X_test = np.delete(X_test, idx_to_remove, axis=2)
            
    
            # Обновление модели
            model = clone_model(self.model_LSTM([X_train.shape[1], X_train.shape[2]] ))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
            
            # Обучение модели на новых данных
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
    
            # Оценка модели на тестовых данных после обучения
            y_pred_after = model.predict(X_test)
            mse_after = mean_squared_error(y_test, y_pred_after)
    
            print(f"Removed feature at index {idx_to_remove}. Test MSE before training: {mse_before}. Test MSE after training: {mse_after}")
    

        column_names = list(data_indicators_pattern.columns)[:-1]  # Последняя колонка "close" не является признаком
                
        # Создание словаря, где ключи - названия колонок, значения - их важность (веса)
        column_importance_dict = dict(zip(column_names, feature_importance[0]))
        print("Важность признаков:")
        for column, importance in column_importance_dict.items():
            print(f"{column}: {importance}")


# Создание экземпляра класса AttentionRFE
rfe = AttentionRFE(data_indicators_pattern)
rfe.fit()

[(1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]


In [134]:


# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Получение градиентов относительно входных данных
with tf.GradientTape() as tape:
    inputs = tf.convert_to_tensor(X_train)
    tape.watch(inputs)
    predictions = model(inputs)
    loss = tf.reduce_mean(tf.square(predictions - y_train))

gradients = tape.gradient(loss, inputs)

# Рассчет абсолютных значений градиентов
absolute_gradients = np.abs(gradients.numpy())
# Усреднение важностей признаков
average_importance = np.mean(absolute_gradients, axis=0)
print(average_importance)
# Предполагаем, что у вас есть массив average_importance
index_of_min_value = np.argmin(average_importance)

index_of_min_value

Epoch 1/10
4/4 [==============================] - 1s 8ms/step - loss: 1.6318
Epoch 2/10
4/4 [==============================] - 0s 0s/step - loss: 1.5756
Epoch 3/10
4/4 [==============================] - 0s 5ms/step - loss: 1.5328
Epoch 4/10
4/4 [==============================] - 0s 0s/step - loss: 1.4893
Epoch 5/10
4/4 [==============================] - 0s 0s/step - loss: 1.4573
Epoch 6/10
4/4 [==============================] - 0s 6ms/step - loss: 1.4186
Epoch 7/10
4/4 [==============================] - 0s 0s/step - loss: 1.3879
Epoch 8/10
4/4 [==============================] - 0s 0s/step - loss: 1.3627
Epoch 9/10
4/4 [==============================] - 0s 6ms/step - loss: 1.3345
Epoch 10/10
4/4 [==============================] - 0s 0s/step - loss: 1.3134


In [139]:
# Рассчет абсолютных значений градиентов
absolute_gradients = np.abs(gradients.numpy())
# Усреднение важностей признаков
average_importance = np.mean(absolute_gradients, axis=0)
print(average_importance)
# Предполагаем, что у вас есть массив average_importance
index_of_min_value = np.argmin(average_importance)

index_of_min_value

[0.0040035  0.00390094 0.00334264 0.00192257 0.00328265 0.00309102
 0.00505311 0.0042888  0.00260999 0.00275602]


3